In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
df = pd.read_csv('Dataset.csv')

In [3]:
df.drop(['Loan Identifier','EMI','UPB_diff','Interest_Component','PREPAYMENT_AMT','Remaining Months To Maturity'], axis=1, inplace = True)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X=df.drop(['Current Actual UPB'],axis=1)
y = df['Current Actual UPB']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
X_train

,Monthly Reporting Period,Channel,Original Interest Rate,Current Interest Rate,Original UPB,Original Loan Term,Origination Date,First Payment Date,Original Loan to Value Ratio (LTV),Debt-To-Income (DTI),...,Modification Flag,Zero Balance Code,Special Eligibility Program,Property Valuation Method,High Balance Loan Indicator,Unemployment_rate,Divorce_rate,Inflation_rate,Previous_UPB,Remaining MFM
745167,12013,1,3.625,3.625,390000.0,360,102012,122012,95,39.0,...,0,0,0,4,0,7.4,3.3,1.6,390000.00,357
175523,22003,1,6.875,6.875,145000.0,360,112001,12002,80,32.0,...,0,0,0,4,0,6.0,3.8,3.0,142787.99,345
532599,32010,2,5.250,5.250,586000.0,360,22009,42009,61,18.0,...,0,0,0,4,0,9.6,3.6,2.3,575914.98,347
412393,32009,0,6.375,6.375,131000.0,360,102006,122006,76,36.0,...,0,0,0,4,0,9.3,3.5,-0.4,127076.54,331
116492,122011,0,6.875,6.875,204000.0,360,112001,12002,95,30.0,...,1,0,0,4,0,8.9,3.6,3.0,200651.15,239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889113,102020,2,4.000,4.000,408000.0,360,12015,42015,80,44.0,...,0,0,0,4,0,8.1,2.3,1.2,364481.11,291
932509,52017,2,5.125,5.125,279000.0,360,92015,112015,90,39.0,...,0,0,0,4,0,4.4,2.9,1.9,272730.51,340
43318,32001,1,8.250,8.250,250000.0,360,121999,22000,66,39.0,...,0,0,0,4,0,4.7,4.0,2.9,250000.00,345
1058110,12020,2,4.625,4.625,175000.0,240,72017,92017,66,38.0,...,0,0,0,0,0,8.1,2.3,2.5,161885.01,210


In [6]:
y_train

745167     390000.00
175523     142654.45
532599     574961.80
412393     127076.54
116492     200651.15
             ...    
889113     363748.20
932509     272375.18
43318      247665.58
1058110    161389.95
595216     107860.96
Name: Current Actual UPB, Length: 1005174, dtype: float64

In [7]:
model = Sequential()

model.add(Dense(units=64, activation='relu', input_dim=X.shape[1]))

model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=16, activation='relu'))

model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_absolute_error')

In [8]:
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32) 

Epoch 1/50
31412/31412 [==============================] - 48s 2ms/step - loss: 6714.9004
Epoch 2/50
31412/31412 [==============================] - 67s 2ms/step - loss: 419.2802
Epoch 3/50
31412/31412 [==============================] - 73s 2ms/step - loss: 372.8446
Epoch 4/50
31412/31412 [==============================] - 90s 3ms/step - loss: 356.8954
Epoch 5/50
31412/31412 [==============================] - 71s 2ms/step - loss: 347.3211
Epoch 6/50
31412/31412 [==============================] - 90s 3ms/step - loss: 343.2314
Epoch 7/50
31412/31412 [==============================] - 87s 3ms/step - loss: 337.7030
Epoch 8/50
31412/31412 [==============================] - 49s 2ms/step - loss: 335.2094
Epoch 9/50
31412/31412 [==============================] - 87s 3ms/step - loss: 330.9719
Epoch 10/50
31412/31412 [==============================] - 63s 2ms/step - loss: 328.4747
Epoch 11/50
31412/31412 [==============================] - 60s 2ms/step - loss: 323.9689
Epoch 12/50
31412/31412 [====

In [9]:
predictions = model.predict(X_test_scaled)

7853/7853 [==============================] - 10s 1ms/step


In [10]:
print(predictions)

[[177276.06 ]
 [161699.11 ]
 [ 78010.7  ]
 ...
 [ 69394.91 ]
 [111289.836]
 [417377.66 ]]


In [11]:
print(y_test)

891706     177307.76
272892     161836.79
405034      78042.24
987251      84886.53
506645     176925.53
             ...    
1021756    110000.00
68061      111644.46
6107        69397.40
1121303    111172.56
1105236    417092.34
Name: Current Actual UPB, Length: 251294, dtype: float64


In [12]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(predictions, y_test)

302.50731495697477

In [13]:
from sklearn.metrics import r2_score
print(r2_score(predictions, y_test))

0.9993398818882518


In [14]:
import pickle
file=open('dnn_a3.pkl','wb')
pickle.dump(model, file)

INFO:tensorflow:Assets written to: ram://64e6777a-2de1-46d4-9d12-eb2b81d3b3b7/assets
